In [1]:
import cv2
import toml
import sys
import os

sys.path.append(os.path.abspath("data_management"))
sys.path.append(os.path.abspath("onnx_models"))

import data_preprocessing as dprep
import data_postprocessing as dpostp
import onnx_inference as onnx_inf

%matplotlib inline

In [2]:
yolo_cfg = toml.load('yolov5n.toml')
imgsz = yolo_cfg['input_data']['imgsz']
conf_thres = yolo_cfg['predictor']['conf_thres']
iou_thres = yolo_cfg['predictor']['iou_thres']
classes = yolo_cfg['predictor']['classes']
if not classes:
    classes = None
onnx_model_path = yolo_cfg['onnx_model_path']
visualize = yolo_cfg['visualize']
out_img_path = yolo_cfg['out_img_path']

In [3]:
stride, names, session, output_names = onnx_inf.load_model(onnx_model_path)

In [4]:
try:
    cap.release()
    cap.destroyAllWindows()
except Exception as e:
    print(e)
    pass

cap = cv2.VideoCapture(0) 

frame_count = 0
while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Check that frame is not empty
    if ret:
        orig_img = frame.copy()
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        img = dprep.img_preprocess(img, imgsz, stride)

        predictions = onnx_inf.model_inf(img, session, output_names)

        predictions = dpostp.yolo_nms(predictions, conf_thres, iou_thres, classes)

        for pred in predictions:

            pred[:, :4] = dpostp.scale_boxes((imgsz, imgsz), pred[:, :4], orig_img.shape).round()

            for p in pred:
                print({"type":int(p[5]) + 1, "x":int(p[0]), "y":int(p[1]), "width":int(p[2])-int(p[0]), "height":int(p[3])-int(p[1])})

                if visualize:
                    cv2.rectangle(orig_img, (int(p[0]), int(p[1])), (int(p[2]), int(p[3])), (0, 255, 0), 2)
                    cv2.putText(orig_img, names[int(p[5])], (int(p[0]), int(p[1])), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        if visualize:
            cv2.imwrite(os.path.join(out_img_path, f'{frame_count}.png'), orig_img)
            frame_count += 1
        


name 'cap' is not defined
{'type': 63, 'x': 23, 'y': 239, 'width': 207, 'height': 238}
{'type': 61, 'x': 479, 'y': 128, 'width': 160, 'height': 347}
{'type': 1, 'x': 448, 'y': 9, 'width': 190, 'height': 457}
{'type': 73, 'x': 332, 'y': 0, 'width': 262, 'height': 467}
{'type': 1, 'x': 477, 'y': 19, 'width': 163, 'height': 451}
{'type': 1, 'x': 444, 'y': 48, 'width': 194, 'height': 422}
{'type': 1, 'x': 398, 'y': 73, 'width': 241, 'height': 407}
{'type': 1, 'x': 390, 'y': 79, 'width': 248, 'height': 401}
{'type': 1, 'x': 336, 'y': 82, 'width': 298, 'height': 398}
{'type': 73, 'x': 39, 'y': 5, 'width': 357, 'height': 450}
{'type': 1, 'x': 295, 'y': 59, 'width': 333, 'height': 421}
{'type': 1, 'x': 260, 'y': 11, 'width': 338, 'height': 469}
{'type': 73, 'x': 5, 'y': 13, 'width': 310, 'height': 458}
{'type': 1, 'x': 254, 'y': 2, 'width': 354, 'height': 478}
{'type': 1, 'x': 246, 'y': 0, 'width': 356, 'height': 480}
{'type': 1, 'x': 236, 'y': 0, 'width': 365, 'height': 480}
{'type': 1, 'x': 

KeyboardInterrupt: 

In [5]:
cap.release()